In [127]:
from pykeops.torch import LazyTensor
import torch
import matplotlib.pyplot as plt

In [128]:
torch.manual_seed(0)

In [129]:
device = 'cuda:1'

In [130]:
def rand_nys_appx(K, n, r, device):
    # Calculate sketch
    Phi = torch.randn((n, r), device=device) / (n ** 0.5)
    Phi = torch.linalg.qr(Phi, mode='reduced')[0]

    Y = K @ Phi

    # Calculate shift
    shift = torch.finfo(Y.dtype).eps
    Y_shifted = Y + shift * Phi

    # Calculate Phi^T * K * Phi (w/ shift) for Cholesky
    choleskytarget = torch.mm(Phi.t(), Y_shifted)

    # Perform Cholesky decomposition
    C = torch.linalg.cholesky(choleskytarget)

    B = torch.linalg.solve_triangular(C, Y_shifted, upper=False, left=False)
    U, S, _ = torch.linalg.svd(B, full_matrices=False)

    return U, S

In [131]:
def loss(K, b, lambd, a):
    return 1/2 * torch.dot(a, K @ a) + lambd / 2 * torch.dot(a, a) \
            - torch.dot(b, a) 

In [132]:
def gd(K, b, lambd, a0, rho, eta, r, max_iter, device):
    n = K.shape[0]
    U, S = rand_nys_appx(K, n, r, device)

    print(S)

    plt.semilogy(S.cpu().numpy())
    plt.show()

    a = a0.clone()
    for i in range(max_iter):
        g = K @ a + lambd * a - b
        UTg = U.t() @ g
        dir = U @ (UTg / (S + rho)) + 1/rho * (g - U @ UTg)
        a -= eta * dir

        print(f"iter {i}, loss {loss(K, b, lambd, a)}")

    return a

In [133]:
def get_blocks(n, B):
    # Permute the indices
    idx = torch.randperm(n)

    # Partition the indices into B blocks of roughly equal size
    # Do this by first computing the block size then making a list of block sizes
    block_size = n // B
    remainder = n % B
    sizes = [block_size] * B

    for i in range(remainder):
        sizes[i] += 1

    blocks = torch.split(idx, sizes)

    return blocks

In [134]:
# For RBF kernel only -- need to generalize
def bcd(x, sigma, b, lambd, a0, rho, eta, B, r, max_iter, device):
    x_i = LazyTensor(x[:, None, :])
    x_j = LazyTensor(x[None, :, :])
    D = ((x_i - x_j) ** 2).sum(dim=2)
    K = (-D / (2 * sigma ** 2)).exp()

    n = K.shape[0]
    blocks = get_blocks(n, B)
    block_preconds = []

    # Compute randomized Nystrom approximation corresponding to each block
    for block in blocks:
        xb_i = LazyTensor(x[block][:, None, :])
        xb_j = LazyTensor(x[block][None, :, :])
        Db = ((xb_i - xb_j) ** 2).sum(dim=2)
        Kb = (-Db / (2 * sigma ** 2)).exp()

        U, S = rand_nys_appx(Kb, block.shape[0], r, device)
        block_preconds.append((U, S))

    a = a0.clone()
    for i in range(max_iter):
        # Randomly select a block
        block_idx = torch.randint(B, (1,))

        # Get the block and its corresponding preconditioner
        block = blocks[block_idx]
        U, S = block_preconds[block_idx]

        # Compute block gradient
        # xb_j = LazyTensor(x[block][None, :, :])
        # DbnT = ((x_i - xb_j) ** 2).sum(dim=2)
        # KbnT = (-DbnT / (2 * sigma ** 2)).exp()

        # gb = n/block.shape[0] * (KbnT @ a[block] + lambd * a[block] - b[block])

        xb_i = LazyTensor(x[block][:, None, :])
        Dbn = ((xb_i - x_j) ** 2).sum(dim=2)
        Kbn = (-Dbn / (2 * sigma ** 2)).exp()

        gb = n/block.shape[0] * (Kbn @ a + lambd * a[block] - b[block])

        # Apply preconditioner
        UTgb = U.t() @ gb
        dir = U @ (UTgb / (S + rho)) + 1/rho * (gb - U @ UTgb)

        # Update block
        a[block] -= eta * dir

        print(f"iter {i}, loss {loss(K, b, lambd, a)}")

In [135]:
# Example sizes
N_x = 1000000  # Number of rows in x

# Generate random data
x = torch.randn(N_x, 3, dtype=torch.float32, requires_grad=True).to(device)

# Define your LazyTensors
x_i = LazyTensor(x[:, None, :])  # Shape (N_x, 1, 3)
x_j = LazyTensor(x[None, :, :])  # Shape (1, N_y, 3)

# Compute the Gaussian kernel matrix
sigma = 1.0  # Kernel width
D_ij = ((x_i - x_j) ** 2).sum(dim=2)  # Squared Euclidean distances
K_ij = (- D_ij / (2 * sigma ** 2)).exp()  # Gaussian kernel matrix, still lazy

In [140]:
b = torch.randn(N_x, dtype=torch.float32).to(device)
lambd = 0.1 #(10 ** -2) / N_x
a0 = torch.zeros(N_x, dtype=torch.float32).to(device)
rho = 1
eta = 0.01
r = 300
max_iter = 1000

In [137]:
# Ground truth
a_star = K_ij.solve(torch.unsqueeze(b, 1), alpha=lambd)
print(loss(K_ij, b, lambd, torch.squeeze(a_star)))

tensor(-4991643.5000, device='cuda:1', grad_fn=<SubBackward0>)


In [138]:
# with torch.no_grad():
#     a = gd(K_ij, b, lambd, a0, rho, eta, r, max_iter, device)

In [142]:
with torch.no_grad():
    a = bcd(x, sigma, b, lambd, a0, rho, eta, 100, r, max_iter, device)

[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.


[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : 